In [1]:
import os
import librosa, librosa.display
import IPython.display as ipd
import math
import json 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./data/features_30_sec.csv')

In [3]:
df

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,rock.00095.wav,661794,0.352063,0.080487,0.079486,0.000345,2008.149458,282174.689224,2106.541053,88609.749506,...,45.050526,-13.289984,41.754955,2.484145,36.778877,-6.713265,54.866825,-1.193787,49.950665,rock
996,rock.00096.wav,661794,0.398687,0.075086,0.076458,0.000588,2006.843354,182114.709510,2068.942009,82426.016726,...,33.851742,-10.848309,39.395096,1.881229,32.010040,-7.461491,39.196327,-2.795338,31.773624,rock
997,rock.00097.wav,661794,0.432142,0.075268,0.081651,0.000322,2077.526598,231657.968040,1927.293153,74717.124394,...,33.597008,-12.845291,36.367264,3.440978,36.001110,-12.588070,42.502201,-2.106337,29.865515,rock
998,rock.00098.wav,661794,0.362485,0.091506,0.083860,0.001211,1398.699344,240318.731073,1818.450280,109090.207161,...,46.324894,-4.416050,43.583942,1.556207,34.331261,-5.041897,47.227180,-3.590644,41.299088,rock


In [4]:
def ae(signal, frame_size, hop_length):
    return np.array([max(signal[i:i+frame_size]) for i in range(0, signal.size, hop_length)])

In [158]:
def save_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=2048,
             hop_length=512, num_segments=5):
    # Data storage dictionary
    data = {
        "mapping": [],
        "mfcc_1": [],
        "mfcc_2": [],
        "mfcc_3": [],
        "mfcc_4": [],
        "mfcc_5": [],
        "mfcc_6": [],
        "mfcc_7": [],
        "mfcc_8": [],
        "mfcc_9": [],
        "mfcc_10": [],
        "chroma_mean": [],
        "chroma_std": [],
        "rms_mean": [],
        "rms_std": [],
        "ae_mean": [],
        "ae_std": [],
        "zcr_mean": [],
        "zcr_std": [],
        "sc_mean": [],
        "sc_std": [],
        "band_mean": [],
        "band_std": [],
        "labels": [],
    }
    samples_ps = int(SAMPLES_PER_TRACK/num_segments) # ps = per segment
    expected_vects_ps = math.ceil(samples_ps/hop_length)
    #
    #count = 0
    # loop through all the genres
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        # ensuring not at root
        
        if dirpath is not dataset_path:
            # save the semantic label
            dirpath_comp = dirpath.split("/")
            semantic_label = dirpath_comp[-1]
            data["mapping"].append(semantic_label)
            print(f"Processing: {semantic_label}")
            count = 0
            # process files for specific genre
            for f in filenames:
                if(f==str("jazz.00054.wav")):
                    # As librosa only read files <1Mb
                    continue
                else:
                    # load audio file
                    file_path = os.path.join(dirpath, f)
                    signal,sr = librosa.load(file_path,sr=SAMPLE_RATE)
                    
                    chroma = librosa.feature.chroma_stft(signal,sr=sr)
                    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=signal, sr=sr, n_fft=n_fft, hop_length=hop_length)[0]
                    spectral_centroid = librosa.feature.spectral_centroid(y=signal, sr=sr, n_fft=n_fft, hop_length=hop_length)[0]
                    zcr = librosa.feature.zero_crossing_rate(signal, frame_length=n_fft, hop_length=hop_length)[0]
                    ae_ = ae(signal, n_fft, hop_length)
                    rms = librosa.feature.rms(signal, frame_length=n_fft, hop_length=hop_length)[0]

                    data["chroma_mean"].append(np.mean(chroma))
                    data["chroma_std"].append(np.std(chroma))
                    data["rms_mean"].append(np.mean(rms))
                    data["rms_std"].append(np.std(rms))
                    data["ae_mean"].append(np.mean(ae_))
                    data["ae_std"].append(np.std(ae_))
                    data["zcr_mean"].append(np.mean(zcr))
                    data["zcr_std"].append(np.std(zcr))
                    data["sc_mean"].append(np.mean(spectral_centroid))
                    data["sc_std"].append(np.std(spectral_centroid))
                    data["band_mean"].append(np.mean(spectral_bandwidth))
                    data["band_std"].append(np.std(spectral_bandwidth))
                    data["labels"].append(i-1)
                    for s in range(num_segments):
                        start_sample = samples_ps * s
                        finish_sample = start_sample + samples_ps

                        mfcc = librosa.feature.mfcc(signal,
                                                    sr = sr,
                                                    n_fft = n_fft,
                                                    n_mfcc = n_mfcc,
                                                    hop_length = hop_length)

                        mfcc = mfcc.T
                        
                        
                        # store featrues 
                        #if len(mfcc)==expected_vects_ps:
                        string = 'mfcc_'+str(s+1)
                        data[string].append(np.mean(mfcc[s]))
                        #print(len(data[string]))
                            #print(f"{file_path}, segment: {s+1}")
                    
                            
                #count +=1
                #if count ==  :break
            #print(len(data["chroma_std"]))
            #print(len(data["rms_mean"]))
    del data['mapping']
    dfobj = pd.DataFrame(data)
    return dfobj

In [159]:
DATASET_PATH = './data/genres_original'
JSON_PATH = 'data2.json'
SAMPLE_RATE = 22050
DURATION = 30
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION
df = save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)

Processing: genres_original\blues
Processing: genres_original\classical
Processing: genres_original\country
Processing: genres_original\disco
Processing: genres_original\hiphop
Processing: genres_original\jazz
Processing: genres_original\metal
Processing: genres_original\pop
Processing: genres_original\reggae
Processing: genres_original\rock


In [160]:
df

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,...,rms_std,ae_mean,ae_std,zcr_mean,zcr_std,sc_mean,sc_std,band_mean,band_std,labels
0,-5.706740,-4.415651,-1.091833,0.681720,3.644113,3.712207,2.797491,1.606959,0.651872,6.021034,...,0.053167,0.375375,0.140416,0.083045,0.027694,1784.165850,360.241675,2002.449060,293.057608,0
1,-9.150007,1.682613,8.894200,9.854056,8.249156,5.946903,5.074986,3.307401,1.777985,-0.543458,...,0.048711,0.288863,0.146209,0.056040,0.038046,1530.176679,613.066125,2039.036516,462.432433,0
2,3.781586,4.258162,4.403440,2.913624,1.507566,1.430652,0.921271,0.191817,0.910968,-1.295847,...,0.052401,0.501379,0.142431,0.076291,0.031731,1552.811865,395.559911,1747.702312,276.141616,0
3,7.084162,6.731664,5.285840,4.890325,4.167927,2.997913,2.814093,2.694005,1.904407,2.091464,...,0.079664,0.344977,0.179490,0.033309,0.020561,1070.106615,429.366909,1596.412872,407.972419,0
4,-16.909956,-12.786216,-12.592538,-13.135061,-14.163464,-13.357795,-6.165907,-3.226989,-3.344306,-4.334365,...,0.047994,0.329738,0.173131,0.101461,0.044205,1835.004266,586.003361,1748.172116,297.397392,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0.400775,-1.950820,-7.695916,-7.552310,-7.292852,-7.816775,-7.661562,-4.700060,-2.503638,-1.024971,...,0.018579,0.246240,0.075453,0.089227,0.036121,2008.149458,531.201176,2106.541053,297.673898,9
995,1.638267,-0.502136,-4.086061,-4.280716,-4.969070,-5.473058,-4.925173,-5.118340,-4.239804,-3.814622,...,0.024253,0.245862,0.092414,0.097664,0.029454,2006.843354,426.749001,2068.942009,287.099315,9
996,-4.131209,-3.784524,-2.548368,-1.603102,-2.815469,-3.741689,-3.820702,-6.226454,-7.538968,-7.905725,...,0.017954,0.263016,0.075060,0.121823,0.041192,2077.526598,481.308600,1927.293153,273.344333,9
997,3.436958,4.617150,4.031875,2.908208,1.277276,-0.212878,-1.490500,-0.958589,-0.353730,-0.274977,...,0.034800,0.237967,0.107956,0.048724,0.028417,1398.699344,490.223144,1818.450280,330.288067,9


In [161]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [162]:
X = df.drop(['labels'],axis=1)
y = df['labels'] 

In [163]:
cols = X.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)

# new data frame with the new scaled data. 
X = pd.DataFrame(np_scaled, columns = cols)

In [164]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=111)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((699, 22), (300, 22), (699,), (300,))

In [165]:
def model_assess(model, title = "Default"):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    #print(confusion_matrix(y_test, preds))
    print('Accuracy', title, ':', round(accuracy_score(y_test, preds), 5), '\n')

In [169]:
# Random Forest Classifier
RFC = RandomForestClassifier()
model_assess(RFC, "RFC")

# Support Vector Machine
svm = SVC(decision_function_shape="ovo")
model_assess(svm, "Support Vector Machine")

# Logistic Regression
lg = LogisticRegression()
model_assess(lg, "Logistic Regression")


Accuracy RFC : 0.58 

Accuracy Support Vector Machine : 0.55 

Accuracy Logistic Regression : 0.5 



C:\Users\homep\Anaconda3\envs\py37\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
